In [ ]:
%pip install bitshuffle
%pip install Pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 99.2 MB/s eta 0:00:00


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import blimpy as bl
#from ultralytics import YOLO
import time
import os
import random
import psutil
import scipy.ndimage
from PIL import Image
from PIL import ImageDraw
from pathlib import Path
import logging
from concurrent.futures import ProcessPoolExecutor, as_completed
import cv2
from scipy.ndimage import label as connected_components
from skimage.filters import sobel
from skimage.segmentation import slic
from skimage.util import img_as_float
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from random import shuffle
from tqdm import tqdm
from skimage.util import img_as_float
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from skimage.segmentation import slic
%matplotlib inline

In [ ]:
df = pd.read_csv('/home/cgchoza/galaxies/complete_cadences_catalog.csv')
df

,Target,Session,Band,Cadence ID,Frequency,.h5 path,.dat path,Time
0,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
1,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
2,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
3,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
4,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
...,...,...,...,...,...,...,...,...
36553,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
36554,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
36555,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
36556,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36


In [ ]:
df['.h5 path'] = df['.h5 path'].str.replace('0000.h5', '0002.h5', regex=False)
df = df.drop_duplicates(subset='.h5 path', keep='first').reset_index(drop=True)
df

,Target,Session,Band,Cadence ID,Frequency,.h5 path,.dat path,Time
0,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
1,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
2,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
3,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
4,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
...,...,...,...,...,...,...,...,...
30309,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
30310,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
30311,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
30312,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36


In [ ]:
df = df[~df['.h5 path'].str.contains('spliced')].reset_index(drop=True)
df

,Target,Session,Band,Cadence ID,Frequency,.h5 path,.dat path,Time
0,AND_XIV,AGBT18B_999_07,S,30225,3151,/datag/pipeline/AGBT18B_999_07/blc40_blp00/blc...,/home/obs/turboseti/AGBT18B_999_07/blc40_blp00...,2018-08-18 08:41:37
1,AND_XIV,AGBT18B_999_07,S,30225,3151,/datag/pipeline/AGBT18B_999_07/blc40_blp00/blc...,/home/obs/turboseti/AGBT18B_999_07/blc40_blp00...,2018-08-18 08:41:37
2,AND_XIV,AGBT18B_999_07,S,30225,3151,/datag/pipeline/AGBT18B_999_07/blc40_blp00/blc...,/home/obs/turboseti/AGBT18B_999_07/blc40_blp00...,2018-08-18 08:41:37
3,AND_XIV,AGBT18B_999_07,S,30225,3151,/datag/pipeline/AGBT18B_999_07/blc40_blp00/blc...,/home/obs/turboseti/AGBT18B_999_07/blc40_blp00...,2018-08-18 08:41:37
4,AND_XIV,AGBT18B_999_07,S,30225,3151,/datag/pipeline/AGBT18B_999_07/blc40_blp00/blc...,/home/obs/turboseti/AGBT18B_999_07/blc40_blp00...,2018-08-18 08:41:37
...,...,...,...,...,...,...,...,...
29341,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
29342,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
29343,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
29344,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36


In [ ]:
df = df.drop(index = 17546)
df = df.sample(n=100, random_state=42).reset_index(drop=True)

In [ ]:
fb = bl.Waterfall(df['.h5 path'].iloc[-1], load_data=True)
fb.info()


--- File Info ---
DIMENSION_LABELS : [b'time' b'feed_id' b'frequency']
        az_start :                              0.0
       data_type :                                1
            fch1 :               10501.46484375 MHz
            foff :         -0.00286102294921875 MHz
           ibeam :                               -1
      machine_id :                               20
          nbeams :                                1
           nbits :                               32
          nchans :                            65536
            nifs :                                1
     rawdatafile : guppi_59411_54386_HIP30264_0094.0000.raw
     source_name :                         HIP30264
         src_dej :                     -8:26:53.228
         src_raj :                      6:21:58.451
    telescope_id :                                6
           tsamp :                1.073741823999999
   tstart (ISOT) :          2021-07-16T15:06:26.000
    tstart (MJD) :                59

In [ ]:
import psutil
import time

def ensure_cpu_mem(bytes_needed, safety=0.8):
    avail = psutil.virtual_memory().available
    if bytes_needed > avail * safety:
        time.sleep(120)

def ensure_gpu_mem(bytes_needed, safety=0.8):
    free, total = cp.cuda.runtime.memGetInfo()
    if bytes_needed > free * safety:
        time.sleep(120)


In [ ]:
def merge_boxes(boxes, iou_thresh=0.1, proximity_thresh=0.05):
    """
    Merge boxes if their centers are close or they have significant IoU overlap.
    boxes: list of (cx, cy, w, h, area)
    returns: merged list of boxes
    """

    def iou(b1, b2):
        x1_min = b1[0] - b1[2] / 2
        x1_max = b1[0] + b1[2] / 2
        y1_min = b1[1] - b1[3] / 2
        y1_max = b1[1] + b1[3] / 2

        x2_min = b2[0] - b2[2] / 2
        x2_max = b2[0] + b2[2] / 2
        y2_min = b2[1] - b2[3] / 2
        y2_max = b2[1] + b2[3] / 2

        inter_xmin = max(x1_min, x2_min)
        inter_ymin = max(y1_min, y2_min)
        inter_xmax = min(x1_max, x2_max)
        inter_ymax = min(y1_max, y2_max)

        inter_area = max(0, inter_xmax - inter_xmin) * max(0, inter_ymax - inter_ymin)
        area1 = b1[2] * b1[3]
        area2 = b2[2] * b2[3]
        union = area1 + area2 - inter_area

        return inter_area / union if union > 0 else 0

    def distance(b1, b2):
        return np.hypot(b1[0] - b2[0], b1[1] - b2[1])

    merged = []
    used = [False] * len(boxes)

    for i, b1 in enumerate(boxes):
        if used[i]:
            continue
        group = [b1]
        used[i] = True
        for j, b2 in enumerate(boxes):
            if used[j]:
                continue
            if iou(b1, b2) > iou_thresh or distance(b1, b2) < proximity_thresh:
                group.append(b2)
                used[j] = True
        if len(group) == 1:
            merged.append(group[0])
        else:
            # merge into one big box (average center, min enclosing width/height)
            xs = [b[0] for b in group]
            ys = [b[1] for b in group]
            ws = [b[2] for b in group]
            hs = [b[3] for b in group]
            xmins = [x - w/2 for x, w in zip(xs, ws)]
            xmaxs = [x + w/2 for x, w in zip(xs, ws)]
            ymins = [y - h/2 for y, h in zip(ys, hs)]
            ymaxs = [y + h/2 for y, h in zip(ys, hs)]
            new_x = (min(xmins) + max(xmaxs)) / 2
            new_y = (min(ymins) + max(ymaxs)) / 2
            new_w = max(xmaxs) - min(xmins)
            new_h = max(ymaxs) - min(ymins)
            merged.append((new_x, new_y, new_w, new_h, new_w * new_h))
    return merged


In [ ]:
# def generate_yolo_boxes(edge_map, threshold=0.1):
#     binary = (edge_map > threshold).astype(np.uint8)
#     labeled, n = connected_components(binary)
#     boxes = []
#     for i in range(1, n + 1):
#         coords = np.argwhere(labeled == i)
#         if coords.shape[0] < 10:
#             continue
#         y0, x0 = coords.min(axis=0)
#         y1, x1 = coords.max(axis=0)
#         cx = (x0 + x1) / 2 / edge_map.shape[1]
#         cy = (y0 + y1) / 2 / edge_map.shape[0]
#         w = (x1 - x0) / edge_map.shape[1]
#         h = (y1 - y0) / edge_map.shape[0]
#         boxes.append((cx, cy, w, h, w * h))
#         return boxes

In [ ]:
def build_tasks(df):
    """
    Build a flat list of (h5_path, channel_idx) from your DataFrame,
    skipping any channels that fall into known notch filter frequency ranges.
    """
    GBT_NOTCH_FILTERS = {
        "L": [(1200, 1340)],
        "S": [(2300, 2360)],
    }

    tasks = []

    for _, row in df.iterrows():
        h5 = row[".h5 path"]
        band = row["Band"]  # e.g., 'L', 'S', etc.
        fb = bl.Waterfall(h5, load_data=False)
        nfreq = fb.header.get("nchans")
        nfpc = fb.header.get("nfpc", 1024)
        fch1 = fb.header["fch1"]
        foff = fb.header["foff"]
        n_coarse = nfreq // nfpc

        for ch in range(n_coarse):
            f0 = fch1 + ch * nfpc * foff
            f1 = fch1 + (ch + 1) * nfpc * foff
            f_min, f_max = sorted([f0, f1])

            # Check against notch filter exclusion ranges
            skip = False
            if band in GBT_NOTCH_FILTERS:
                for lo, hi in GBT_NOTCH_FILTERS[band]:
                    if lo <= f_min <= hi or lo <= f_max <= hi:
                        skip = True
                        break
            if not skip:
                tasks.append((h5, ch))

    return tasks


def split_tasks(tasks, train_frac=0.8, seed=42):
    """
    Shuffle & split the flat task list into train vs. val sets.
    Returns two sets of (h5_path, channel_idx).
    """
    random.seed(seed)
    shuffled = tasks.copy()
    random.shuffle(shuffled)
    cut = int(train_frac * len(shuffled))
    train = set(shuffled[:cut])
    val   = set(shuffled[cut:])
    return train, val

In [ ]:
# import cv2
# import numpy as np
# import torch
# from torch import nn
# from pathlib import Path
# from PIL import Image, ImageDraw
# import blimpy as bl
# from skimage.segmentation import slic
# #from yolo_moe_pytorch_channel import generate_yolo_boxes

# # ─── 1) Define your rule-based gate ──────────────────────────────────────
# class RuleGate(nn.Module):
#     def __init__(self, w_align=2.0, w_ent=1.0, bias=-0.5):
#         super().__init__()
#         # simple fixed scalars (or wrap in nn.Parameter to fine‐tune later)
#         self.w_align = w_align
#         self.w_ent   = w_ent
#         self.bias    = bias

#     def forward(self, h_sf, h_ued):
#         # h_sf, h_ued are torch.tensors of shape [R,3] or [R,4]
#         # index 1 = alignment, 2 = entropy
#         align = h_ued[:,1] - h_sf[:,1]       # higher → favor Sobel
#         ent   = h_sf[:,2] - h_ued[:,2]       # higher → favor Sobel
#         score  = self.w_align*align + self.w_ent*ent + self.bias
#         return torch.sigmoid(score)          # [R] weights for Sobel

# # ─── 2) Heuristic helper (no change) ────────────────────────────────────
# def compute_heuristics(edge_map, gray):
#     mask = edge_map > 0.2
#     density = float(mask.mean())
#     # alignment
#     if density > 0:
#         gx = cv2.Sobel(gray, cv2.CV_32F, 1, 0, 3)
#         gy = cv2.Sobel(gray, cv2.CV_32F, 0, 1, 3)
#         alignment = float(np.hypot(gx, gy)[mask].mean())
#     else:
#         alignment = 0.0
#     # entropy
#     entropy = float(-np.sum(edge_map * np.log2(edge_map + 1e-8)))
#     # linearity (aspect‐ratio proxy)
#     bw = mask.astype(np.uint8)
#     n, labels = cv2.connectedComponents(bw)
#     ratios = []
#     for L in range(1, n):
#         ys, xs = np.where(labels == L)
#         if ys.size:
#             h, w = ys.ptp()+1, xs.ptp()+1
#             ratios.append(float(w)/h if h else 0.0)
#     linearity = float(np.mean(ratios)) if ratios else 0.0

#     if density < 0.01:
#         return [0.0, 0.0, 0.0, 0.0]
#     return [density, alignment, entropy, linearity]


# # ─── 3) The new process_file ─────────────────────────────────────────────
# def process_file(job):
#     (h5_path, channels, gidxs,
#      base_dir, factor, dilation,
#      class_id, train_set, val_set,
#      pad_width) = job

#     # Load .h5 once
#     fb   = bl.Waterfall(h5_path, load_data=True)
#     data = 10*np.log10(fb.data.squeeze())  # (ntime, nfreq)

#     # Prepare SF detector and gate
#     EDGE_MODEL = "/home/jliang/gbt-rfi/model.yml.gz"
#     sf_det     = cv2.ximgproc.createStructuredEdgeDetection(EDGE_MODEL)
#     gate       = RuleGate()  # use default w_align/w_ent/bias

#     for ch_idx, gidx in zip(channels, gidxs):
#         # [.. create img_dir, lbl_dir, vis_dir ..]
#         subset = "train" if (h5_path,ch_idx) in train_set else "val"
#         img_dir = Path(base_dir)/subset/"images"
#         lbl_dir = Path(base_dir)/subset/"labels"
#         vis_dir = Path(base_dir)/"visualization"/subset
#         for d in (img_dir,lbl_dir,vis_dir): d.mkdir(parents=True, exist_ok=True)

#         # Extract & clean the block
#         cw = fb.header.get("nfpc",1024)
#         f0, f1 = ch_idx*cw, (ch_idx+1)*cw
#         block = data[:,f0:f1]
#         low, high = int(0.1*cw), int(0.9*cw)
#         block = block[:,low:high]
#         h_img, w_img = block.shape

#         # Build img for SF
#         norm   = (block - block.min())/(block.ptp()+1e-6)
#         img3   = np.stack([norm]*3, axis=-1).astype(np.float32)
#         sf_map = sf_det.detectEdges(img3).squeeze()
#         gray8  = (255*norm).astype(np.uint8)

#         # Sobel map
#         gx      = cv2.Sobel(gray8, cv2.CV_32F, 1,0,3)
#         gy      = cv2.Sobel(gray8, cv2.CV_32F, 0,1,3)
#         ued_map = np.hypot(gx, gy)
#         ued_map = cv2.normalize(ued_map, None, 0,1, cv2.NORM_MINMAX)

#         # Fuse per-superpixel or per-pixel
#         fused = np.zeros_like(sf_map, dtype=np.float32)
#         segments = slic(img3, n_segments=100, compactness=10)
#         for v in np.unique(segments):
#             mask = (segments==v)
#             if mask.sum()<50:
#                 continue
#             # compute heuristics for this region
#             gray8 = (255*norm).astype(np.uint8)
#             h_sf  = compute_heuristics(sf_map[mask], gray8[mask])
#             h_ued = compute_heuristics(ued_map[mask], gray8[mask])
#             h_sf_t  = torch.tensor([h_sf], dtype=torch.float32)
#             h_ued_t = torch.tensor([h_ued],dtype=torch.float32)
#             p = gate(h_sf_t, h_ued_t).item()  # how much to trust Sobel
#             fused[mask] = p*ued_map[mask] + (1-p)*sf_map[mask]

#         # YOLO boxes
#         boxes = generate_yolo_boxes(fused, threshold=0.1)
#         # [.. save PNG, write .txt, draw visualization exactly as before ..]
#         # Calculate frequency range of this coarse channel
#         f_start = fb.header['fch1'] + f0 * fb.header['foff']
#         f_stop  = fb.header['fch1'] + (f1 - 1) * fb.header['foff']

#         # Build filename
#         fn = f"img_{gidx:0{pad_width}d}_f_{f_start:.4f}_{f_stop:.4f}.png"
#         img_path = img_dir / fn
#         txt_path = lbl_dir / fn.replace(".png", ".txt")

#         # Save image
#         arr8 = (255 * (block - block.min()) / (block.ptp() + 1e-6)).astype(np.uint8)
#         img = Image.fromarray(np.stack([arr8]*3, axis=-1))
#         img.save(img_path)

#         # Write YOLO labels
#         with open(txt_path, "w") as f:
#             for x_c, y_c, w_n, h_n in boxes:
#                 f.write(f"{class_id} {x_c:.6f} {y_c:.6f} {w_n:.6f} {h_n:.6f}\n")

#         # If empty, log it
#         if not boxes:
#             (lbl_dir/"empty_labels.csv").open("a").write(f"{fn},{h5_path},{ch_idx}\n")

#         # Draw & save visualization
#         vis_img = img.convert("RGB")
#         draw    = ImageDraw.Draw(vis_img)
#         for x_c, y_c, w_n, h_n in boxes:
#             xc, yc = x_c*w_img, y_c*h_img
#             bw, bh = w_n*w_img, h_n*h_img
#             x0, y0 = int(xc - bw/2), int(yc - bh/2)
#             x1, y1 = int(xc + bw/2), int(yc + bh/2)
#             draw.rectangle([x0, y0, x1, y1], outline="red", width=2)
#         vis_img.save(vis_dir/fn)



In [ ]:
# import cv2
# import numpy as np
# import torch
# from torch import nn
# from pathlib import Path
# from PIL import Image, ImageDraw
# import blimpy as bl
# from skimage.segmentation import slic
# from skimage.filters import threshold_sauvola
# import scipy.ndimage
# from collections import namedtuple
# from scipy.stats import kurtosis

# # ─── 0) BoundingBox, IOU & Merge ──────────────────────────────────────────
# BoundingBox = namedtuple("BoundingBox",
#     ["x_c","y_c","w","h","area","class_id"]
# )

# def calculate_iou(b1, b2):
#     x1min, x1max = b1.x_c - b1.w/2, b1.x_c + b1.w/2
#     y1min, y1max = b1.y_c - b1.h/2, b1.y_c + b1.h/2
#     x2min, x2max = b2.x_c - b2.w/2, b2.x_c + b2.w/2
#     y2min, y2max = b2.y_c - b2.h/2, b2.y_c + b2.h/2

#     xi0 = max(x1min, x2min); yi0 = max(y1min, y2min)
#     xi1 = min(x1max, x2max); yi1 = min(y1max, y2max)
#     if xi1<=xi0 or yi1<=yi0:
#         return 0.0
#     inter = (xi1-xi0)*(yi1-yi0)
#     union = b1.w*b1.h + b2.w*b2.h - inter
#     return inter/union if union>0 else 0.0

# def merge_boxes(boxes, iou_thresh=0.1, prox_thresh=0.05):
#     used = [False]*len(boxes)
#     merged = []
#     for i,b in enumerate(boxes):
#         if used[i]: continue
#         group = [b]; used[i]=True
#         for j in range(i+1,len(boxes)):
#             if used[j]: continue
#             bj = boxes[j]
#             dist = np.hypot(b.x_c-bj.x_c, b.y_c-bj.y_c)
#             if calculate_iou(b,bj)>iou_thresh or dist<prox_thresh:
#                 group.append(bj); used[j]=True
#         if len(group)==1:
#             merged.append(b)
#         else:
#             xmins = [g.x_c-g.w/2 for g in group]
#             xmaxs = [g.x_c+g.w/2 for g in group]
#             ymins = [g.y_c-g.h/2 for g in group]
#             ymaxs = [g.y_c+g.h/2 for g in group]
#             new_w = max(xmaxs)-min(xmins)
#             new_h = max(ymaxs)-min(ymins)
#             new_x = (min(xmins)+max(xmaxs))/2
#             new_y = (min(ymins)+max(ymaxs))/2
#             new_area = new_w*new_h
#             merged.append(BoundingBox(new_x,new_y,new_w,new_h,new_area,group[0].class_id))
#     return merged

# # ─── 1) Improved YOLO box generator ───────────────────────────────────────
# def generate_yolo_boxes(edge_map,
#                         threshold=0.1,
#                         min_pixels=20,
#                         iou_thresh=0.1,
#                         prox_thresh=0.05):
#     H,W = edge_map.shape

#     # (a) Knock out persistent horizontal RFI bands
#     bw = (edge_map>threshold).astype(np.uint8)
#     horiz_kernel = np.ones((1,50),np.uint8)
#     band_mask = scipy.ndimage.binary_opening(bw, structure=horiz_kernel)
#     clean = edge_map.copy()
#     clean[band_mask] = np.median(edge_map)

#     # (b) Local Sauvola threshold
#     th = threshold_sauvola(clean, window_size=51, k=0.3)
#     mask = clean > th

#     # (c) Morphological cleanup
#     mask = scipy.ndimage.binary_opening(mask, structure=np.ones((5,5)))
#     mask = scipy.ndimage.binary_closing(mask, structure=np.ones((1,30)))
#     mask = scipy.ndimage.binary_closing(mask, structure=np.ones((30,1)))

#     # (d) CC → raw boxes
#     labeled, n = scipy.ndimage.label(mask)
#     boxes=[]
#     for lab in range(1,n+1):
#         ys,xs = np.where(labeled==lab)
#         if ys.size<min_pixels: continue
#         y0,y1 = ys.min(), ys.max()
#         x0,x1 = xs.min(), xs.max()
#         w = (x1-x0)/W; h=(y1-y0)/H
#         x_c=(x0+(x1-x0)/2)/W; y_c=(y0+(y1-y0)/2)/H
#         area=w*h
#         boxes.append(BoundingBox(x_c,y_c,w,h,area, class_id=0))

#     # (e) Merge clusters
#     boxes = merge_boxes(boxes, iou_thresh, prox_thresh)
#     return boxes

# # ─── 2) Your RuleGate & compute_heuristics (unchanged) ────────────────────
# class RuleGate(nn.Module):
#     def __init__(self, w_align=2.0, w_ent=1.0, bias=-0.5):
#         super().__init__()
#         self.w_align, self.w_ent, self.bias = w_align, w_ent, bias
#     def forward(self, h_sf, h_ued):
#         align = h_ued[:,1]-h_sf[:,1]
#         ent   = h_sf[:,2]-h_ued[:,2]
#         return torch.sigmoid(self.w_align*align + self.w_ent*ent + self.bias)

# def compute_heuristics(edge_map, gray):
#     mask = edge_map>0.2
#     density = float(mask.mean())
#     if density>0:
#         gx=cv2.Sobel(gray,cv2.CV_32F,1,0,3)
#         gy=cv2.Sobel(gray,cv2.CV_32F,0,1,3)
#         alignment=float(np.hypot(gx,gy)[mask].mean())
#     else:
#         alignment=0.0
#     entropy = float(-np.sum(edge_map*np.log2(edge_map+1e-8)))
#     bw=mask.astype(np.uint8)
#     n,labs=cv2.connectedComponents(bw)
#     ratios=[]
#     for L in range(1,n):
#         ys,xs=np.where(labs==L)
#         if ys.size:
#             ratios.append((np.ptp(xs)+1)/(np.ptp(ys)+1))
#     linearity=float(np.mean(ratios)) if ratios else 0.0
#     return [density, alignment, entropy, linearity] if density>=0.01 else [0,0,0,0]

# # ─── 3) Updated process_file ──────────────────────────────────────────────
# def process_file(job):
#     (h5_path, channels, gidxs,
#      base_dir,
#      class_id, train_set, val_set,
#      pad_width, file_idx) = job

#     fb = bl.Waterfall(h5_path, load_data=True)
#     data = 10*np.log10(fb.data.squeeze())
#     nt,nf = data.shape
#     nfpc = fb.header.get("nfpc",1024)

#     # SF detector & RuleGate
#     EDGE_MODEL="/home/jliang/gbt-rfi/model.yml.gz"
#     sf_det = cv2.ximgproc.createStructuredEdgeDetection(EDGE_MODEL)
#     gate   = RuleGate()

#     for ch_idx, gidx in zip(channels, gidxs):
#         subset = "train" if (h5_path,ch_idx) in train_set else "val"
#         img_dir=Path(base_dir)/subset/"images"
#         lbl_dir=Path(base_dir)/subset/"labels"
#         vis_dir=Path(base_dir)/"visualization"/subset
#         for d in (img_dir,lbl_dir,vis_dir): d.mkdir(parents=True,exist_ok=True)

#         cw = fb.header.get("nfpc",1024)
#         f0, f1 = ch_idx*cw, (ch_idx+1)*cw
#         block = data[:,f0:f1]
#         low, high = int(0.15*cw), int(0.85*cw)
#         block = block[:,low:high]
#         H, W = block.shape

#         # Remove vertical line artifact
#         rows, cols = block.shape
#         vert_means = block.mean(axis=0)
#         center = np.argmax(vert_means)
#         left_col = center - 1
#         right_col = center + 1
#         if left_col >= 0 and right_col < cols:
#             block[:, center] = (block[:, left_col] + block[:, right_col]) / 2
#         elif left_col >= 0:
#             block[:, center] = block[:, left_col]
#         elif right_col < cols:
#             block[:, center] = block[:, right_col]

#         # normalize
#         norm=(block-block.min())/(np.ptp(block)+1e-8)
#         img3=np.stack([norm]*3,axis=-1).astype(np.float32)

#         # SF map
#         sf_map=sf_det.detectEdges(img3).squeeze()
#         gray8=(norm*255).astype(np.uint8)

#         # Sobel map
#         gx=cv2.Sobel(gray8,cv2.CV_32F,1,0,3)
#         gy=cv2.Sobel(gray8,cv2.CV_32F,0,1,3)
#         ued_map=np.hypot(gx,gy)
#         ued_map=cv2.normalize(ued_map,None,0,1,cv2.NORM_MINMAX)

#         # fuse per-superpixel
#         fused=np.zeros_like(sf_map,dtype=np.float32)
#         segments=slic(img3,n_segments=100,compactness=10)
#         for seg in np.unique(segments):
#             m=segments==seg
#             if m.sum()<50: continue
#             h_sf=compute_heuristics(sf_map[m], gray8[m])
#             h_ued=compute_heuristics(ued_map[m],gray8[m])
#             p=gate(torch.tensor([h_sf]),torch.tensor([h_ued])).item()
#             fused[m]=p*ued_map[m]+(1-p)*sf_map[m]

#         # generate & save YOLO boxes
#         boxes=generate_yolo_boxes(fused,
#                                   threshold=0.2,
#                                   min_pixels=30,
#                                   iou_thresh=0.1,
#                                   prox_thresh=0.05)

#         k_val = kurtosis(gray8.flatten(), fisher=True, bias=False)
#         f_start = fb.header['fch1'] + f0 * fb.header['foff']
#         f_stop  = fb.header['fch1'] + (f1 - 1) * fb.header['foff']
#         fn = f"img_{k_val:.3f}_f_{f_start:.4f}_{f_stop:.4f}.png"
#         arr8=(norm*255).astype(np.uint8)
#         img=Image.fromarray(arr8).convert("RGB")
#         img.save(img_dir/fn)

#         with open(lbl_dir/fn.replace(".png",".txt"),"w") as f:
#             for b in boxes:
#                 f.write(f"{class_id} {b.x_c:.6f} {b.y_c:.6f} {b.w:.6f} {b.h:.6f}\n")
#         if not boxes:
#             (lbl_dir/"empty_labels.csv").open("a").write(f"{fn},{h5_path},{ch_idx}\n")

#         vis=img.copy(); draw=ImageDraw.Draw(vis)
#         for b in boxes:
#             x0=int((b.x_c-b.w/2)*W); y0=int((b.y_c-b.h/2)*H)
#             x1=int((b.x_c+b.w/2)*W); y1=int((b.y_c+b.h/2)*H)
#             draw.rectangle([x0,y0,x1,y1],outline="red",width=2)
#         vis.save(vis_dir/fn)

#     logging.info(f"Processed {h5_path}")

# if __name__ == "__main__":
#     logging.basicConfig(level=logging.INFO)

#     # 1) your tasks must be list of (h5_path, ch_idx)
#     tasks = build_tasks(df)
#     train_set, val_set = split_tasks(tasks)

#     BASE_DIR    = '/datax/scratch/jliang/dataset_moe_local_thresholding-v2'
#     NUM_WORKERS = 4

#     # 2) pad_width for zero-padding file_idx
#     total_images = len(tasks)
#     pad_width    = len(str(total_images))

#     # 3) build jobs, now unpacks cleanly in process_file
#     job_list = []
#     for file_idx, (h5_path, ch_idx) in enumerate(tasks):
#         job = (
#             h5_path,
#             [ch_idx],       # channels
#             [file_idx],     # gidxs (or whatever you want)
#             BASE_DIR,
#             0,              # class_id (default)
#             train_set,
#             val_set,
#             pad_width,
#             file_idx        # <-- now process_file will get this too
#         )
#         job_list.append(job)

#     # 4) submit and catch errors
#     with ProcessPoolExecutor(max_workers=NUM_WORKERS) as exe:
#         futures = [exe.submit(process_file, job) for job in job_list]
#         for fut in tqdm(as_completed(futures), total=len(futures)):
#             try:
#                 fut.result()   # raise any exceptions here
#             except Exception as e:
#                 logging.error(f"Error in job: {e}")

#     logging.info("All tasks completed.")


In [ ]:
import logging
import cv2
import numpy as np
import torch
from torch import nn
from pathlib import Path
from PIL import Image, ImageDraw
import blimpy as bl
from skimage.segmentation import slic
from skimage.filters import threshold_sauvola
import scipy.ndimage
from scipy.stats import kurtosis
from astropy.stats import sigma_clip
import scipy.signal
from collections import namedtuple
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm
import pandas as pd
import random

# ─── 0) Task builders ─────────────────────────────────────────────────────
def build_tasks(df):
    tasks = []
    for _, row in df.iterrows():
        h5 = row[".h5 path"]
        fb = bl.Waterfall(h5, load_data=False)
        nfreq = fb.header.get("nchans")
        nfpc  = fb.header.get("nfpc", 1024)
        n_coarse = nfreq // nfpc
        for ch in range(n_coarse):
            tasks.append((h5, ch))
    return tasks

def split_tasks(tasks, train_frac=0.8, seed=42):
    random.seed(seed)
    sh = tasks.copy()
    random.shuffle(sh)
    cut = int(train_frac * len(sh))
    return set(sh[:cut]), set(sh[cut:])

# ─── 1) BoundingBox, IOU & Merge ──────────────────────────────────────────
BoundingBox = namedtuple("BoundingBox",
    ["x_c","y_c","w","h","area","class_id"]
)

def calculate_iou(b1, b2):
    x1min, x1max = b1.x_c - b1.w/2, b1.x_c + b1.w/2
    y1min, y1max = b1.y_c - b1.h/2, b1.y_c + b1.h/2
    x2min, x2max = b2.x_c - b2.w/2, b2.x_c + b2.w/2
    y2min, y2max = b2.y_c - b2.h/2, b2.y_c + b2.h/2

    xi0 = max(x1min, x2min); yi0 = max(y1min, y2min)
    xi1 = min(x1max, x2max); yi1 = min(y1max, y2max)
    if xi1 <= xi0 or yi1 <= yi0:
        return 0.0
    inter = (xi1-xi0)*(yi1-yi0)
    union = b1.w*b1.h + b2.w*b2.h - inter
    return inter/union if union>0 else 0.0

def merge_boxes(boxes, iou_thresh=0.1, prox_thresh=0.05):
    used = [False]*len(boxes)
    merged = []
    for i,b in enumerate(boxes):
        if used[i]: continue
        group = [b]; used[i]=True
        for j in range(i+1, len(boxes)):
            if used[j]: continue
            bj = boxes[j]
            dist = np.hypot(b.x_c - bj.x_c, b.y_c - bj.y_c)
            if calculate_iou(b,bj)>iou_thresh or dist<prox_thresh:
                group.append(bj); used[j]=True
        if len(group)==1:
            merged.append(b)
        else:
            xmins = [g.x_c - g.w/2 for g in group]
            xmaxs = [g.x_c + g.w/2 for g in group]
            ymins = [g.y_c - g.h/2 for g in group]
            ymaxs = [g.y_c + g.h/2 for g in group]
            new_w = max(xmaxs) - min(xmins)
            new_h = max(ymaxs) - min(ymins)
            new_x = (min(xmins) + max(xmaxs)) / 2
            new_y = (min(ymins) + max(ymaxs)) / 2
            new_area = new_w * new_h
            merged.append(BoundingBox(new_x, new_y, new_w, new_h, new_area, group[0].class_id))
    return merged

# ─── 1.1) Spectral SNR‐based verifier (borrowed) ────────────────────────────
def snr_bounds(spec, snr=5):
    """
    Returns (l, r, metadata) if a peak exceeds snr×noise_std,
    else raises ValueError.
    """
    peak = np.argmax(spec)
    y = sigma_clip(spec)
    x = np.arange(len(spec))

    coeffs = np.polyfit(x[~y.mask], y[~y.mask], 1)
    poly = np.poly1d(coeffs)
    std = np.std(y[~y.mask] - poly(x[~y.mask]))

    widths, heights, left_ips, right_ips = scipy.signal.peak_widths(
        spec, [peak], 1 - (snr * std) / np.max(spec)
    )
    if np.isnan(left_ips[0]) or np.isnan(right_ips[0]):
        raise ValueError("No peak above SNR threshold")
    l = int(np.floor(left_ips[0])) + 1
    r = int(np.ceil(right_ips[0]))

    return l, r, {"std": std, "spec_max": np.max(spec)}

def verify_box(intensity_map, box, snr_min=5):
    H, W = intensity_map.shape
    x0 = int((box.x_c - box.w/2) * W)
    x1 = int((box.x_c + box.w/2) * W)
    # collapse freq-slice
    slice_ = intensity_map[:, max(x0,0):min(x1,W)].mean(axis=1)
    try:
        snr_bounds(slice_, snr=snr_min)
        return True
    except ValueError:
        return False

# ─── 2) YOLO‐style Box Generator with solidity & spectral verification ──
def generate_yolo_boxes(edge_map,
                        intensity_map,
                        thresh_factor=3.5,
                        min_pixels=30,
                        max_frac=0.6,
                        kurtosis_thresh=0.7,
                        solidity_thresh=0.5,
                        iou_thresh=0.1,
                        prox_thresh=0.05,
                        snr_thresh=6):
    H, W = edge_map.shape

    # (a) remove horiz bands
    bw = (edge_map>0).astype(np.uint8)
    horiz = np.ones((1, W//4), np.uint8)
    band_mask = scipy.ndimage.binary_opening(bw, structure=horiz)
    clean = edge_map.copy()
    clean[band_mask] = np.median(edge_map)

    # (b) global threshold
    M, S = np.median(clean), np.std(clean)
    mask = clean > (M + thresh_factor * S)

    # (c) Sauvola
    sauv = threshold_sauvola(clean, window_size=61, k=0.3)
    mask |= (clean > sauv)

    # (d) morphology
    mask = scipy.ndimage.binary_opening(mask, structure=np.ones((3,3)))
    mask = scipy.ndimage.binary_closing(mask, structure=np.ones((1,20)))
    mask = scipy.ndimage.binary_closing(mask, structure=np.ones((20,1)))

    # (e) CC → raw boxes with solidity & kurtosis
    labeled, n = scipy.ndimage.label(mask)
    boxes = []
    for lab in range(1, n+1):
        ys, xs = np.where(labeled==lab)
        if ys.size < min_pixels:
            continue
        y0, y1 = ys.min(), ys.max()
        x0, x1 = xs.min(), xs.max()
        w_frac = (x1 - x0) / W
        h_frac = (y1 - y0) / H
        if w_frac>max_frac or h_frac>max_frac:
            continue
        ar = w_frac/h_frac if h_frac>0 else 999
        if ar>8 or ar<0.125:
            continue

        # solidity filter
        region_mask = (labeled[y0:y1+1, x0:x1+1]==lab).astype(np.uint8)
        contours, _ = cv2.findContours(region_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if not contours:
            continue
        cnt = contours[0]
        hull = cv2.convexHull(cnt)
        hull_area = cv2.contourArea(hull)
        region_area = region_mask.sum()
        solidity = region_area / (hull_area + 1e-8)
        if solidity < solidity_thresh:
            continue

        # kurtosis filter
        patch = intensity_map[y0:y1+1, x0:x1+1].ravel()
        if kurtosis(patch, fisher=True, bias=False) < kurtosis_thresh:
            continue

        x_c = (x0 + (x1-x0)/2) / W
        y_c = (y0 + (y1-y0)/2) / H
        area = w_frac * h_frac
        boxes.append(BoundingBox(x_c, y_c, w_frac, h_frac, area, class_id=0))

    # (f) merge
    boxes = merge_boxes(boxes, iou_thresh, prox_thresh)
    # (g) spectral SNR verification
    boxes = [b for b in boxes if verify_box(intensity_map, b, snr_min=snr_thresh)]
    return boxes

# ─── 3) RuleGate & Heuristics ─────────────────────────────────────────────
class RuleGate(nn.Module):
    def __init__(self, w_align=2.0, w_ent=1.0, bias=-0.5):
        super().__init__()
        self.w_align, self.w_ent, self.bias = w_align, w_ent, bias
    def forward(self, h_sf, h_ued):
        align = h_ued[:,1] - h_sf[:,1]
        ent   = h_sf[:,2] - h_ued[:,2]
        return torch.sigmoid(self.w_align*align + self.w_ent*ent + self.bias)

def compute_heuristics(edge_map, gray):
    mask = edge_map > 0.2
    density = float(mask.mean())
    if density>0:
        gx = cv2.Sobel(gray,cv2.CV_32F,1,0,3)
        gy = cv2.Sobel(gray,cv2.CV_32F,0,1,3)
        alignment = float(np.hypot(gx,gy)[mask].mean())
    else:
        alignment = 0.0
    entropy = float(-np.sum(edge_map * np.log2(edge_map + 1e-8)))
    bw = mask.astype(np.uint8)
    n, labs = cv2.connectedComponents(bw)
    ratios = []
    for L in range(1, n):
        ys, xs = np.where(labs==L)
        if ys.size:
            ratios.append((np.ptp(xs)+1)/(np.ptp(ys)+1))
    linearity = float(np.mean(ratios)) if ratios else 0.0
    return [density, alignment, entropy, linearity] if density>=0.01 else [0,0,0,0]

# ─── 4) process_file ──────────────────────────────────────────────────────
def process_file(job):
    (h5_path, channels, gidxs,
     base_dir, class_id,
     train_set, val_set,
     pad_width, file_idx) = job

    fb   = bl.Waterfall(h5_path, load_data=True)
    data = 10 * np.log10(fb.data.squeeze())  # (ntime, nfreq)
    nt, nf = data.shape
    nfpc = fb.header.get("nfpc", 1024)

    EDGE_MODEL = "/home/jliang/gbt-rfi/model.yml.gz"
    sf_det     = cv2.ximgproc.createStructuredEdgeDetection(EDGE_MODEL)
    gate       = RuleGate()

    for ch_idx, gidx in zip(channels, gidxs):
        subset = "train" if (h5_path,ch_idx) in train_set else "val"
        img_dir = Path(base_dir)/subset/"images"
        lbl_dir = Path(base_dir)/subset/"labels"
        vis_dir = Path(base_dir)/"visualization"/subset
        for d in (img_dir,lbl_dir,vis_dir):
            d.mkdir(parents=True, exist_ok=True)

        f0, f1 = ch_idx*nfpc, (ch_idx+1)*nfpc
        block  = data[:, f0:f1]
        low, high = int(0.15*nfpc), int(0.85*nfpc)
        block = block[:, low:high]
        H, W  = block.shape
        f_start = fb.header['fch1'] + f0 * fb.header['foff']
        f_stop  = fb.header['fch1'] + (f1 - 1) * fb.header['foff']


        # Remove vertical line artifact
        rows, cols = block.shape
        vert_means = block.mean(axis=0)
        center = np.argmax(vert_means)
        left_col = center - 1
        right_col = center + 1

        if left_col >= 0 and right_col < cols:
            block[:, center] = (block[:, left_col] + block[:, right_col]) / 2
        elif left_col >= 0:
            block[:, center] = block[:, left_col]
        elif right_col < cols:
            block[:, center] = block[:, right_col]


        arr8_orig = ((block - block.min())/(np.ptp(block)+1e-8)*255).astype(np.uint8)
        img_orig  = Image.fromarray(arr8_orig).convert("RGB")

        norm = (block - block.min())/(np.ptp(block)+1e-8)
        img3 = np.stack([norm]*3, axis=-1).astype(np.float32)

        sf_map = sf_det.detectEdges(img3).squeeze()
        gray8  = arr8_orig

        gx = cv2.Sobel(gray8,cv2.CV_32F,1,0,3)
        gy = cv2.Sobel(gray8,cv2.CV_32F,0,1,3)
        ued_map = np.hypot(gx,gy)
        ued_map = cv2.normalize(ued_map,None,0,1,cv2.NORM_MINMAX)

        fused = np.zeros_like(sf_map, dtype=np.float32)
        segments = slic(img3, n_segments=200, compactness=5)
        for seg in np.unique(segments):
            m = segments==seg
            if m.sum()<50: continue
            h_sf  = compute_heuristics(sf_map[m], gray8[m])
            h_ued = compute_heuristics(ued_map[m], gray8[m])
            w     = gate(torch.tensor([h_sf]), torch.tensor([h_ued])).item()
            fused[m] = w*ued_map[m] + (1-w)*sf_map[m]

        boxes = generate_yolo_boxes(
            fused,
            intensity_map=norm,
            thresh_factor=3.5,
            min_pixels=30,
            max_frac=0.6,
            kurtosis_thresh=0.7,
            solidity_thresh=0.5,
            iou_thresh=0.1,
            prox_thresh=0.05,
            snr_thresh=6
        )

        k_val = kurtosis(gray8.flatten(), fisher=True, bias=False)
        fn = f"img_{k_val:.3f}_f_{f_start:.3f}_{f_stop:.3f}.png"

        img_orig.save(img_dir/fn)

        with open(lbl_dir/fn.replace(".png",".txt"), "w") as f:
            for b in boxes:
                f.write(f"{class_id} {b.x_c:.6f} {b.y_c:.6f} {b.w:.6f} {b.h:.6f}\n")
        if not boxes:
            (lbl_dir/"empty_labels.csv").open("a").write(f"{fn},{h5_path},{ch_idx}\n")

        vis = img_orig.copy()
        draw = ImageDraw.Draw(vis)
        for b in boxes:
            x0 = int((b.x_c - b.w/2)*W)
            y0 = int((b.y_c - b.h/2)*H)
            x1 = int((b.x_c + b.w/2)*W)
            y1 = int((b.y_c + b.h/2)*H)
            draw.rectangle([x0,y0,x1,y1], outline="red", width=2)
        vis.save(vis_dir/fn)

    logging.info(f"Processed {h5_path}")

# ─── 5) Main Entrypoint ────────────────────────────────────────────────────
if __name__ == "__main__":
    logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s [%(levelname)s] %(message)s",
        datefmt="%Y-%m-%d %H:%M:%S",
    )

    tasks    = build_tasks(df)
    train_set, val_set = split_tasks(tasks, train_frac=0.8, seed=42)

    BASE_DIR    = "/datax/scratch/jliang/dataset_updated"
    NUM_WORKERS = 8
    pad_width   = len(str(len(tasks)-1))

    job_list = []
    for file_idx, (h5_path, ch_idx) in enumerate(tasks):
        job = (
            h5_path,
            [ch_idx],
            [file_idx],
            BASE_DIR,
            0,
            train_set,
            val_set,
            pad_width,
            file_idx
        )
        job_list.append(job)

    with ProcessPoolExecutor(max_workers=NUM_WORKERS) as exe:
        futures = [exe.submit(process_file, j) for j in job_list]
        for _ in tqdm(as_completed(futures), total=len(futures)):
            try:
                _.result()  # raise any exceptions here
            except Exception as e:
                logging.error(f"Error in job: {e}")

    logging.info("All tasks completed.")

In [ ]:
# import numpy as np
# import cv2
# import torch
# import torch.nn as nn
# from skimage.segmentation import slic
# from scipy import ndimage

# # ─── 1) HEURISTIC COMPUTATION ─────────────────────────────────────────

# def compute_heuristics(edge_map, gray_image):
#     # edge_map: 2D float32 [0,1], gray_image: 2D uint8
#     # 1) gradient magnitude
#     gx = cv2.Sobel(gray_image, cv2.CV_32F, 1, 0, ksize=3)
#     gy = cv2.Sobel(gray_image, cv2.CV_32F, 0, 1, ksize=3)
#     grad = np.hypot(gx, gy)

#     # 2) statistics
#     density   = np.mean(edge_map > 0.2)
#     align     = np.mean(grad[edge_map > 0.2]) if density>0 else 0.0
#     entropy   = -np.sum(edge_map * np.log2(edge_map + 1e-8))
#     # 3) linearity via eccentricity of components
#     bw   = edge_map > 0.2
#     lbl, n = ndimage.label(bw)
#     props = ndimage.find_objects(lbl)
#     eccs = []
#     for i, sl in enumerate(props, start=1):
#         region = (lbl[sl] == i).astype(np.uint8)
#         # approximate by second moments → skip details here
#         eccs.append(region.sum()>0 and region.sum() / (region.shape[0]*region.shape[1]))
#     linearity = float(np.mean(eccs)) if eccs else 0.0

#     if density < 0.01:
#         return [0.0, 0.0, 0.0, 0.0]
#     return [density, align, entropy, linearity]


# # ─── 2) RULE-BASED GATE MODULE ──────────────────────────────────────────

# class RuleGate(nn.Module):
#     def __init__(self, w_align=1.0, w_ent=1.0, bias=0.0):
#         super().__init__()
#         # wrap as parameters if you want to tune via backprop:
#         self.w_align = nn.Parameter(torch.tensor(w_align))
#         self.w_ent   = nn.Parameter(torch.tensor(w_ent))
#         self.bias    = nn.Parameter(torch.tensor(bias))

#     def forward(self, h_sf, h_ued):
#         # h_*: [B,4] tensors on CUDA
#         align = h_ued[:,1] - h_sf[:,1]      # favors UED if >0
#         ent   = h_sf[:,2] - h_ued[:,2]      # favors UED if >0
#         score  = self.w_align * align + self.w_ent * ent + self.bias
#         return torch.sigmoid(score)         # [B] weights for UED


# class MoEBlock(nn.Module):
#     def __init__(self, gate: RuleGate):
#         super().__init__()
#         self.gate = gate

#     def forward(self, sf_map, ued_map, h_sf, h_ued):
#         """
#         sf_map, ued_map: [B,H,W] floats on CUDA
#         h_sf, h_ued:        [B,4] heuristic tensors on CUDA
#         """
#         p = self.gate(h_sf, h_ued).view(-1,1,1)  # [B,1,1]
#         return p * ued_map + (1 - p) * sf_map


# # ─── 3) FULL PROCESSING LOOP ───────────────────────────────────────────

# def process_image_patch(patch, sf_detector, gate_block, device='cuda'):
#     """
#     patch: HxWx3 BGR uint8
#     sf_detector: cv2.ximgproc StructuredEdgeDetection
#     gate_block:   instance of MoEBlock on device
#     """
#     # 1) compute both edge maps
#     gray    = cv2.cvtColor(patch, cv2.COLOR_BGR2GRAY)
#     sf_edges= sf_detector.detectEdges(np.float32(patch)/255.0).squeeze()
#     gx = cv2.Sobel(gray, cv2.CV_32F, 1,0,3)
#     gy = cv2.Sobel(gray, cv2.CV_32F, 0,1,3)
#     ued_edges = np.hypot(gx, gy)
#     ued_edges = (ued_edges/ued_edges.max()).astype(np.float32)

#     # 2) superpixel segmentation
#     segments = slic(patch, n_segments=100, compactness=10)

#     # 3) gather heuristics and masks
#     feats_sf, feats_ued, masks = [], [], []
#     for seg in np.unique(segments):
#         mask = (segments==seg)
#         if mask.sum()<50: continue
#         hsf = compute_heuristics(sf_edges*mask, gray)
#         hud = compute_heuristics(ued_edges*mask, gray)
#         if not any(hsf) and not any(hud): continue
#         feats_sf.append(hsf)
#         feats_ued.append(hud)
#         masks.append(mask)

#     if not masks:
#         return np.zeros_like(sf_edges, np.float32)

#     # 4) move to GPU
#     h_sf = torch.tensor(feats_sf, device=device)
#     h_ued= torch.tensor(feats_ued, device=device)
#     sf_m = torch.tensor(np.stack([sf_edges[m] for m in masks]), device=device)
#     ue_m = torch.tensor(np.stack([ued_edges[m] for m in masks]), device=device)

#     # 5) fuse with gate
#     fused_masks = gate_block(sf_m, ue_m, h_sf, h_ued)  # [N_pixels]
#     fused = np.zeros_like(sf_edges, np.float32)
#     for p, m in zip(fused_masks.cpu().numpy(), masks):
#         fused[m] = p

#     # 6) generate YOLO boxes
#     return fused


# # ─── 4) USAGE ─────────────────────────────────────────────────────────

# # Initialize:
# EDGE_MODEL_PATH = "model.yml"  # your SF model
# sf_detector = cv2.ximgproc.createStructuredEdgeDetection(EDGE_MODEL_PATH)
# gate        = RuleGate(w_align=2.0, w_ent=1.0, bias=-0.5).cuda()
# moe_block   = MoEBlock(gate).cuda()

# # For each patch:
# # fused_map = process_image_patch(patch, sf_detector, moe_block)
# # then threshold fused_map, label, convert to YOLO (cx,cy,w,h) as before.


In [ ]:
# def compute_heuristics(image, edge_map):
#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     gradient = sobel(gray)
#     edge_density = np.mean(edge_map > 0.2)
#     edge_gradient_alignment = np.mean(gradient[edge_map > 0.2])
#     edge_entropy = -np.sum(edge_map * np.log2(edge_map + 1e-8))
#     lin = linearity_score(edge_map)
#     if edge_density < 0.01:
#         return [0.0, 0.0, 0.0, 0.0]
#     return [edge_density, edge_gradient_alignment, edge_entropy, lin]

# def pseudo_labels(image, sf_edges, ued_edges):
#     h_sf  = compute_heuristics(image, sf_edges)
#     h_ued = compute_heuristics(image, ued_edges)
#     # if UED better alignment & lower entropy, choose UED
#     if h_ued[1] > h_sf[1] and h_ued[2] < h_sf[2]:
#         return 0
#     else:
#         return 1

# # PyTorch gating network (input_dim=8 for 4 heuristics each)
# class GatingNet(nn.Module):
#     def __init__(self, input_dim=8, hidden=16):
#         super().__init__()
#         self.net = nn.Sequential(
#             nn.Linear(input_dim, hidden),
#             nn.ReLU(),
#             nn.Linear(hidden, 2)
#         )
#     def forward(self, x):
#         return self.net(x)

# # Train gating model in PyTorch
# def train_gating_model_torch(images, sf_edge_maps, ued_edge_maps,
#                              batch_size=64, epochs=10, lr=1e-3,
#                              device='cuda'):
#     # 1) Collect features & labels
#     feats, labels = [], []
#     for img, sf, ued in zip(images, sf_edge_maps, ued_edge_maps):
#         segments = slic(img_as_float(img), n_segments=100, compactness=10)
#         for seg_val in np.unique(segments):
#             mask = (segments == seg_val)
#             if mask.sum() < 50:
#                 continue
#             sf_patch, ued_patch = sf * mask, ued * mask
#             if sf_patch.mean() + ued_patch.mean() < 0.01:
#                 continue
#             h_sf  = compute_heuristics(img, sf_patch)
#             h_ued = compute_heuristics(img, ued_patch)
#             feats.append(np.array(h_sf + h_ued, dtype=np.float32))
#             labels.append(pseudo_labels(img, sf_patch, ued_patch))
#     if len(set(labels)) < 2:
#         return None, None, None

#     # 2) Build tensors & standardize
#     X = torch.tensor(feats)  # (N, 8)
#     y = torch.tensor(labels, dtype=torch.long)  # (N,)
#     mean, std = X.mean(dim=0, keepdim=True), X.std(dim=0, keepdim=True) + 1e-6
#     X = (X - mean) / std

#     # 3) DataLoader
#     dataset = TensorDataset(X, y)
#     loader  = DataLoader(dataset, batch_size=batch_size, shuffle=True)

#     # 4) Model, loss, optimizer
#     model = GatingNet(input_dim=X.shape[1]).to(device)
#     optimizer = optim.Adam(model.parameters(), lr=lr)
#     criterion = nn.CrossEntropyLoss()

#     # 5) Training loop
#     model.train()
#     for epoch in range(epochs):
#         total_loss = 0.0
#         for xb, yb in loader:
#             xb, yb = xb.to(device), yb.to(device)
#             logits = model(xb)
#             loss   = criterion(logits, yb)
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()
#             total_loss += loss.item()
#         print(f"Epoch {epoch+1}/{epochs} - loss: {total_loss/len(loader):.4f}")

#     return model, mean.to(device), std.to(device)

# # Apply gating model to fuse edges
# def apply_gating_model_torch(image, sf_edges, ued_edges,
#                              model, mean, std, device='cuda'):
#     segments = slic(img_as_float(image), n_segments=100, compactness=10)
#     feats, masks = [], []
#     for seg_val in np.unique(segments):
#         mask = (segments == seg_val)
#         if mask.sum() < 50:
#             continue
#         sf_patch, ued_patch = sf_edges * mask, ued_edges * mask
#         if sf_patch.mean() + ued_patch.mean() < 0.01:
#             continue
#         h_sf, h_ued = compute_heuristics(image, sf_patch), compute_heuristics(image, ued_patch)
#         f = np.array(h_sf + h_ued, dtype=np.float32)
#         if not np.all(np.isfinite(f)) or np.allclose(f, 0, atol=1e-3):
#             continue
#         feats.append(f)
#         masks.append(mask)

#     if not feats:
#         return np.zeros_like(sf_edges, dtype=float)

#     X = torch.tensor(feats).to(device)
#     X = (X - mean) / std
#     model.eval()
#     with torch.no_grad():
#         probs = torch.softmax(model(X), dim=1)[:,1].cpu().numpy()

#     gated = np.zeros_like(sf_edges, dtype=float)
#     for p, mask in zip(probs, masks):
#         gated[mask] = p * sf_edges[mask] + (1 - p) * ued_edges[mask]
#     return gated


In [ ]:
# def edge_detection(gray, factor=6, dilation=(3, 3)):
#         # STEP 1: apply power threshold to suppress background
#         median = np.median(gray)
#         mad = np.median(np.abs(gray - median))
#         power_mask = gray > (median + factor * mad)

#         # STEP 2: edge detection on filtered signal only
#         grad_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
#         grad_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
#         magnitude = np.sqrt(grad_x**2 + grad_y**2)
#         magnitude *= power_mask  # mask out noise
#         magnitude = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

#         # STEP 3: binary threshold + dilation
#         binary = magnitude > 50  # you can tune this
#         binary = scipy.ndimage.binary_dilation(binary, structure=np.ones(dilation))
#         labeled, n_objs = scipy.ndimage.label(binary)
#         slices = scipy.ndimage.find_objects(labeled)
#         return slices

# EDGE_MODEL_PATH = '/home/jliang/gbt-rfi/model.yml.gz'
# EDGE_DETECTOR = cv2.ximgproc.createStructuredEdgeDetection(EDGE_MODEL_PATH)

# def detect_edges(spectrogram):
#     img = (spectrogram - spectrogram.min()) / (spectrogram.ptp() + 1e-6)
#     img_3ch = cv2.merge([img.astype(np.float32)] * 3)
#     return EDGE_DETECTOR.detectEdges(img_3ch)

In [ ]:
# def extract_one_sample(h5_path, ch, edge_model_path='/home/jliang/gbt-rfi/model.yml.gz'):
#     try:
#         EDGE_DETECTOR = cv2.ximgproc.createStructuredEdgeDetection(edge_model_path)
#         fb = bl.Waterfall(h5_path, load_data=True)
#         data = 10 * np.log10(fb.data.squeeze())

#         nfpc = fb.header.get("nfpc", 1024)
#         f0, f1 = ch * nfpc, (ch + 1) * nfpc
#         block = data[:, f0:f1]

#         n_cols = block.shape[1]
#         low, high = int(0.1 * n_cols), int(0.9 * n_cols)
#         block_middle80 = block[:, low:high]

#         vert_means = block_middle80.mean(axis=0)
#         center = np.argmax(vert_means)
#         left_col, right_col = center - 1, center + 1
#         if left_col >= 0 and right_col < block_middle80.shape[1]:
#             block_middle80[:, center] = (block_middle80[:, left_col] + block_middle80[:, right_col]) / 2
#         elif left_col >= 0:
#             block_middle80[:, center] = block_middle80[:, left_col]
#         elif right_col < block_middle80.shape[1]:
#             block_middle80[:, center] = block_middle80[:, right_col]

#         img_norm = (block_middle80 - block_middle80.min()) / (block_middle80.ptp() + 1e-6)
#         img_rgb = cv2.merge([img_norm.astype(np.float32)] * 3)

#         sf_edges = EDGE_DETECTOR.detectEdges(img_rgb)

#         gray = (255 * img_norm).astype(np.uint8)
#         grad_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
#         grad_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
#         magnitude = np.sqrt(grad_x**2 + grad_y**2)
#         magnitude = cv2.normalize(magnitude, None, 0, 1, cv2.NORM_MINMAX)

#         sf_mean = np.mean(sf_edges)
#         ued_mean = np.mean(magnitude)

#         if sf_mean + ued_mean < 0.005:  # threshold to skip empty or background-only blocks
#             return None  # skip empty or background-only blocks


#         return (img_rgb, sf_edges, magnitude)

#     except Exception as e:
#         print(f"Error processing {h5_path} ch {ch}: {e}")
#         return None

# def extract_samples_batched(df, total_samples=40000, batch_size=32, num_workers=4):
#     all_tasks = []
#     for _, row in df.iterrows():
#         h5 = row[".h5 path"]
#         fb = bl.Waterfall(h5, load_data=False)
#         nfreq = fb.header.get("nchans")
#         nfpc = fb.header.get("nfpc", 1024)
#         for ch in range(nfreq // nfpc):
#             all_tasks.append((h5, ch))

#     shuffle(all_tasks)
#     selected_tasks = all_tasks[:total_samples]

#     train_images, train_sf_edges, train_ued_edges = [], [], []
#     for i in tqdm(range(0, total_samples, batch_size)):
#         batch = selected_tasks[i:i+batch_size]
#         with ProcessPoolExecutor(max_workers=num_workers) as executor:
#             futures = [executor.submit(extract_one_sample, h5, ch) for h5, ch in batch]
#             for future in as_completed(futures):
#                 result = future.result()
#                 if result:
#                     img_rgb, sf_edge, ued_edge = result
#                     train_images.append(img_rgb)
#                     train_sf_edges.append(sf_edge)
#                     train_ued_edges.append(ued_edge)

#     return train_images, train_sf_edges, train_ued_edges



In [ ]:
# # 4) SET UP LOGGING
# logging.basicConfig(
#     level=logging.INFO,
#     format="%(asctime)s [%(levelname)s] %(message)s",
#     datefmt="%Y-%m-%d %H:%M:%S",
# )

# from gating_model import train_gating_model_torch

# def build_tasks(df):
#     """
#     Build a flat list of (h5_path, channel_idx) from your DataFrame,
#     skipping any channels that fall into known notch filter frequency ranges.
#     """
#     GBT_NOTCH_FILTERS = {
#         "L": [(1200, 1340)],
#         "S": [(2300, 2360)],
#     }

#     tasks = []

#     for _, row in df.iterrows():
#         h5 = row[".h5 path"]
#         band = row["Band"]  # e.g., 'L', 'S', etc.
#         fb = bl.Waterfall(h5, load_data=False)
#         nfreq = fb.header.get("nchans")
#         nfpc = fb.header.get("nfpc", 1024)
#         fch1 = fb.header["fch1"]
#         foff = fb.header["foff"]
#         n_coarse = nfreq // nfpc

#         for ch in range(n_coarse):
#             f0 = fch1 + ch * nfpc * foff
#             f1 = fch1 + (ch + 1) * nfpc * foff
#             f_min, f_max = sorted([f0, f1])

#             # Check against notch filter exclusion ranges
#             skip = False
#             if band in GBT_NOTCH_FILTERS:
#                 for lo, hi in GBT_NOTCH_FILTERS[band]:
#                     if lo <= f_min <= hi or lo <= f_max <= hi:
#                         skip = True
#                         break
#             if not skip:
#                 tasks.append((h5, ch))

#     return tasks


# def split_tasks(tasks, train_frac=0.8, seed=42):
#     """
#     Shuffle & split the flat task list into train vs. val sets.
#     Returns two sets of (h5_path, channel_idx).
#     """
#     random.seed(seed)
#     shuffled = tasks.copy()
#     random.shuffle(shuffled)
#     cut = int(train_frac * len(shuffled))
#     train = set(shuffled[:cut])
#     val   = set(shuffled[cut:])
#     return train, val

# def process_file(job):
#     """
#     job is a tuple:
#       (h5_path, channels, global_indices,
#        base_dir, factor, dilation,
#        class_id, train_set, val_set,
#        pad_width, model, mean, std)
#     """
#     (h5_path, channels, global_indices,
#      base_dir, factor, dilation,
#      class_id, train_set, val_set,
#      pad_width, model, mean, std) = job

#     # 1) Prepare device & trained gate
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     model = model.to(device)
#     model.eval()

#     # 2) Load Structured Forest detector once
#     EDGE_MODEL_PATH = "/home/jliang/gbt-rfi/model.yml.gz"
#     sf_detector = cv2.ximgproc.createStructuredEdgeDetection(EDGE_MODEL_PATH)

#     # 3) Load the .h5 file
#     fb   = bl.Waterfall(h5_path, load_data=True)
#     data = 10 * np.log10(fb.data.squeeze())   # (ntime, nfreq)

#     # 4) Process each coarse channel
#     for ch_idx, gidx in zip(channels, global_indices):
#         # decide train vs val
#         subset = "train" if (h5_path, ch_idx) in train_set else "val"

#         # make sure directories exist
#         img_dir = Path(base_dir)/subset/"images"
#         lbl_dir = Path(base_dir)/subset/"labels"
#         vis_dir = Path(base_dir)/"visualization"/subset
#         for d in (img_dir, lbl_dir, vis_dir):
#             d.mkdir(parents=True, exist_ok=True)

#         # extract the 80% middle of the block
#         cw = fb.header.get("nfpc", 1024)
#         f0, f1 = ch_idx*cw, (ch_idx+1)*cw
#         block = data[:, f0:f1]
#         low, high = int(0.1*cw), int(0.9*cw)
#         block = block[:, low:high]

#         # remove single‐column artifact
#         col_means = block.mean(axis=0)
#         c = int(np.argmax(col_means))
#         if 0 < c < block.shape[1]-1:
#             block[:,c] = 0.5*(block[:,c-1] + block[:,c+1])

#         # normalize & build RGB input for SF
#         norm = (block - block.min())/(block.ptp()+1e-6)
#         img_rgb = cv2.merge([norm.astype(np.float32)]*3)

#         # compute the two expert edge‐maps
#         sf_edge_map  = sf_detector.detectEdges(img_rgb).squeeze()
#         gray_uint8  = (255*norm).astype(np.uint8)
#         gx = cv2.Sobel(gray_uint8, cv2.CV_32F, 1, 0, ksize=3)
#         gy = cv2.Sobel(gray_uint8, cv2.CV_32F, 0, 1, ksize=3)
#         ued_edge_map = np.hypot(gx, gy)
#         ued_edge_map = cv2.normalize(ued_edge_map, None, 0.0, 1.0, cv2.NORM_MINMAX)

#         # 5) Fuse using the PyTorch gate
#         fused_edges = apply_gating_model_torch(
#             img_rgb, sf_edge_map, ued_edge_map,
#             model, mean, std,
#             device=device
#         )

#         # 6) Generate & save YOLO boxes
#         boxes = generate_yolo_boxes(fused_edges, threshold=0.1)

#         # Dimensions for filtering
#         h_img, w_img = block.shape
#         min_size = 3                   # px
#         full_image_threshold = 0.95    # normalized area

#         # Filter
#         final_boxes = []
#         for x_c, y_c, w_n, h_n, area in boxes:
#             if area > full_image_threshold:
#                 continue
#             if (w_n * w_img) < min_size or (h_n * h_img) < min_size:
#                 continue
#             final_boxes.append((x_c, y_c, w_n, h_n))

#         # Calculate frequency range of this coarse channel
#         f_start = fb.header['fch1'] + f0 * fb.header['foff']
#         f_stop  = fb.header['fch1'] + (f1 - 1) * fb.header['foff']

#         # Build filename
#         fn = f"img_{gidx:0{pad_width}d}_f_{f_start:.4f}_{f_stop:.4f}.png"
#         img_path = img_dir / fn
#         txt_path = lbl_dir / fn.replace(".png", ".txt")

#         # Save image
#         arr8 = (255 * (block - block.min()) / (block.ptp() + 1e-6)).astype(np.uint8)
#         img = Image.fromarray(np.stack([arr8]*3, axis=-1))
#         img.save(img_path)

#         # Write YOLO labels
#         with open(txt_path, "w") as f:
#             for x_c, y_c, w_n, h_n in final_boxes:
#                 f.write(f"{class_id} {x_c:.6f} {y_c:.6f} {w_n:.6f} {h_n:.6f}\n")

#         # If empty, log it
#         if not final_boxes:
#             (lbl_dir/"empty_labels.csv").open("a").write(f"{fn},{h5_path},{ch_idx}\n")

#         # Draw & save visualization
#         vis_img = img.convert("RGB")
#         draw    = ImageDraw.Draw(vis_img)
#         for x_c, y_c, w_n, h_n in final_boxes:
#             xc, yc = x_c*w_img, y_c*h_img
#             bw, bh = w_n*w_img, h_n*h_img
#             x0, y0 = int(xc - bw/2), int(yc - bh/2)
#             x1, y1 = int(xc + bw/2), int(yc + bh/2)
#             draw.rectangle([x0, y0, x1, y1], outline="red", width=2)
#         vis_img.save(vis_dir/fn)



# if __name__ == "__main__":
#     # === your settings ===
#     BASE_DIR             = "/datax/scratch/jliang/dataset_moe"
#     FACTOR              = 6
#     DILATION = (30, 30)
#     CLASS_ID             = 0
#     TRAIN_FRAC           = 0.8
#     SEED                 = 42
#     NUM_WORKERS          = 2

#     # --- assume you already have a DataFrame `df` with at least 'h5_path' and optionally 'nchans' ---
#     # df = pd.read_csv(...)  # or however you built it

#     # 1) Build & split tasks
#     tasks = build_tasks(df)
#     train_set, val_set = split_tasks(tasks, TRAIN_FRAC, SEED)

#     # 2) compute zero-pad width from total images
#     pad_width = len(str(len(tasks) - 1))

#     # 3) regroup tasks by file to load each .h5 only once
#     jobs = {}
#     for gidx, (h5, ch) in enumerate(tasks):
#         jobs.setdefault(h5, {"chs": [], "gidxs": []})
#         jobs[h5]["chs"].append(ch)
#         jobs[h5]["gidxs"].append(gidx)

#     # 4) prepare job‐tuples
#     job_list = []
#     for h5, info in jobs.items():
#         job_list.append((
#             h5,
#             info["chs"],
#             info["gidxs"],
#             BASE_DIR,
#             FACTOR,
#             DILATION,
#             CLASS_ID,
#             train_set,
#             val_set,
#             pad_width
#         ))

#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#     # train_images, train_sf_edges, train_ued_edges = extract_samples_batched(
#     #     df, total_samples=40000, num_workers=2
#     # )

#     # model, mean, std = train_gating_model_torch(
#     #     train_images,
#     #     train_sf_edges,
#     #     train_ued_edges,
#     #     batch_size=64,
#     #     epochs=10,
#     #     lr=1e-3,
#     #     device=device
#     # )

#     # if model is None:
#     #     logging.warning("Skipping inference because gate net couldn't train.")
#     #     exit()


In [ ]:
# import h5py, numpy as np, torch, logging
# from torch.utils.data import IterableDataset, DataLoader
# import cv2
# from skimage.segmentation import slic
# from yolo_moe_pytorch_channel import (
#     apply_gating_model_torch,
#     generate_yolo_boxes,
#     compute_heuristics,
#     pseudo_labels
# )
# from PIL import Image, ImageDraw

# # 1) Streaming dataset
# class H5ChannelDataset(IterableDataset):
#     def __init__(self, df, nfpc=1024):
#         self.paths = df[".h5 path"].tolist()
#         self.nfpc  = nfpc
#     def __iter__(self):
#         for p in self.paths:
#             with h5py.File(p, "r") as f:
#                 d = f["data"]
#                 n_coarse = d.shape[1] // self.nfpc
#                 for ch in range(n_coarse):
#                     yield d[:, ch*self.nfpc:(ch+1)*self.nfpc].astype(np.float32)

# # 2) Pre- and post-processing helpers
# EDGE_MODEL_PATH = "/home/jliang/gbt-rfi/model.yml.gz"
# sf_detector    = cv2.ximgproc.createStructuredEdgeDetection(EDGE_MODEL_PATH)

# def preprocess(block):
#     norm = (block - block.min())/(block.ptp()+1e-6)
#     img_rgb = np.stack([norm]*3, axis=-1).astype(np.float32)
#     sf_map   = sf_detector.detectEdges(img_rgb).squeeze()
#     gray8    = (255*norm).astype(np.uint8)
#     gx = cv2.Sobel(gray8, cv2.CV_32F, 1,0,3)
#     gy = cv2.Sobel(gray8, cv2.CV_32F, 0,1,3)
#     ued_map = np.hypot(gx, gy)
#     ued_map = cv2.normalize(ued_map, None, 0.0, 1.0, cv2.NORM_MINMAX)
#     return img_rgb, sf_map, ued_map

# def segment_patches(img, sf, ued):
#     segs = slic(img, n_segments=100, compactness=10)
#     for v in np.unique(segs):
#         m = segs==v
#         if m.sum()<50: continue
#         sf_p, ued_p = sf*m, ued*m
#         if sf_p.mean()+ued_p.mean()<0.01: continue
#         yield sf_p, ued_p

# # 3) Collate: build feature / label tensors
# def collate_fn(batch):
#     feats, labs = [], []
#     for block in batch:
#         img, sf, ued = preprocess(block)
#         for sf_p, ued_p in segment_patches(img, sf, ued):
#             h_sf = compute_heuristics(sf_p, img)
#             h_ued= compute_heuristics(ued_p, img)
#             feats.append(h_sf + h_ued)
#             labs.append(pseudo_labels(img, sf_p, ued_p))
#     if not feats: return None
#     return torch.tensor(feats), torch.tensor(labs)

# # 4) Training
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# dataset = H5ChannelDataset(df)
# loader  = DataLoader(dataset,
#                      batch_size=4,
#                      num_workers=4,
#                      persistent_workers=True,
#                      collate_fn=collate_fn)

# from your_module import GatingNet  # the BatchNorm version
# net   = GatingNet().to(device)
# opt   = torch.optim.Adam(net.parameters(), lr=1e-3)
# crit  = nn.CrossEntropyLoss()

# for epoch in range(10):
#     for batch in loader:
#         if batch is None: continue
#         X, y = batch
#         X, y = X.to(device), y.to(device)
#         logits = net(X)
#         loss   = crit(logits, y)
#         opt.zero_grad()
#         loss.backward()
#         opt.step()
#     print(f"Epoch {epoch+1}/10 done")

# # 5) Build job_list as before, but append (net) instead of (mean,std)
# # and in process_file call apply_gating_model_torch(img, sf, ued, net, device=...)

# # 6) Inference uses your existing process_file, no further change needed.


# # ─── 3) Build job list for inference ────────────────────────────────────

# # your existing build_tasks(), split_tasks(), jobs grouping...
# tasks    = build_tasks(df)
# train_set, val_set = split_tasks(tasks, TRAIN_FRAC, SEED)
# pad_width = len(str(len(tasks)))

# jobs = {}
# for gidx,(h5,ch) in enumerate(tasks):
#     jobs.setdefault(h5, {"chs":[], "gidxs":[]})
#     jobs[h5]["chs"].append(ch)
#     jobs[h5]["gidxs"].append(gidx)

# job_list = []
# for h5,info in jobs.items():
#     job_list.append((h5,
#                      info["chs"],
#                      info["gidxs"],
#                      BASE_DIR,
#                      FACTOR,
#                      DILATION,
#                      CLASS_ID,
#                      train_set,
#                      val_set,
#                      pad_width,
#                      gate_net, mean, std))

# # ─── 4) Run inference in parallel ───────────────────────────────────────
# from concurrent.futures import ProcessPoolExecutor

# with ProcessPoolExecutor(max_workers=NUM_WORKERS) as exe:
#     for _ in exe.map(process_file, job_list):
#         pass
